# 🎓 Week 10 - Day 2: CNN Architectures
## From Pooling to Famous Networks in PyTorch

### Today's Learning Goals:
- ✅ Master pooling layers (Max and Average)
- ✅ Understand complete CNN building blocks
- ✅ Build CNNs from scratch in PyTorch
- ✅ Implement VGG-style architecture
- ✅ Understand ResNet skip connections
- ✅ Train a CNN on Fashion-MNIST
- ✅ Visualize learned filters and feature maps
- ✅ Compare different architectures

---

**Let's build real CNNs that actually work!** 🚀

## 📦 Part 1: Setup and Data Loading

Let's import libraries and load Fashion-MNIST dataset!

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

print(f"✅ PyTorch version: {torch.__version__}")
print("✅ All libraries imported successfully!")

In [ ]:
# Load Fashion-MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

# Training data
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

# Test data
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f"\n✅ Dataset loaded successfully!")
print(f"📊 Training samples: {len(train_dataset):,}")
print(f"📊 Test samples: {len(test_dataset):,}")
print(f"📊 Number of classes: {len(class_names)}")
print(f"📊 Image shape: {train_dataset[0][0].shape}")

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.flatten()

for i in range(10):
    img, label = train_dataset[i]
    img = img.squeeze().numpy()
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(f"{class_names[label]}", fontsize=10, fontweight='bold')
    axes[i].axis('off')

plt.suptitle("Fashion-MNIST Sample Images", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Fashion-MNIST: 10 categories of clothing and accessories!")

## 🌊 Part 2: Understanding Pooling Layers

Let's visualize how pooling reduces spatial dimensions!

In [ ]:
# Create a sample feature map
sample_feature = torch.randn(1, 1, 8, 8)

# Apply max pooling (2x2, stride=2)
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
max_pooled = max_pool(sample_feature)

# Apply average pooling (2x2, stride=2)
avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
avg_pooled = avg_pool(sample_feature)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Original
im1 = axes[0].imshow(sample_feature[0, 0].numpy(), cmap='viridis')
axes[0].set_title(f"Original Feature Map\n{sample_feature.shape[2]}×{sample_feature.shape[3]}", 
                  fontsize=12, fontweight='bold')
plt.colorbar(im1, ax=axes[0])
axes[0].grid(True, color='white', linewidth=1)

# Max pooled
im2 = axes[1].imshow(max_pooled[0, 0].numpy(), cmap='viridis')
axes[1].set_title(f"Max Pooled (2×2)\n{max_pooled.shape[2]}×{max_pooled.shape[3]}", 
                  fontsize=12, fontweight='bold')
plt.colorbar(im2, ax=axes[1])
axes[1].grid(True, color='white', linewidth=1)

# Average pooled
im3 = axes[2].imshow(avg_pooled[0, 0].numpy(), cmap='viridis')
axes[2].set_title(f"Average Pooled (2×2)\n{avg_pooled.shape[2]}×{avg_pooled.shape[3]}", 
                  fontsize=12, fontweight='bold')
plt.colorbar(im3, ax=axes[2])
axes[2].grid(True, color='white', linewidth=1)

plt.tight_layout()
plt.show()

print("\n🌊 Pooling Effects:")
print(f"• Original shape: {sample_feature.shape}")
print(f"• After pooling: {max_pooled.shape}")
print(f"• Spatial size reduced: 8×8 → 4×4 (75% reduction!)")
print(f"\n💡 Max pooling keeps strongest activations!")
print(f"💡 Average pooling smooths the output!")

## 🏗️ Part 3: Building Your First Complete CNN

Let's build a simple CNN with all the components!

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 28×28 → 28×28
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 14×14 → 14×14
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Halves the size
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Conv Block 1: Conv → ReLU → Pool
        x = self.pool(F.relu(self.conv1(x)))  # 28×28 → 14×14
        
        # Conv Block 2: Conv → ReLU → Pool
        x = self.pool(F.relu(self.conv2(x)))  # 14×14 → 7×7
        
        # Flatten for FC layers
        x = x.view(-1, 64 * 7 * 7)  # Flatten
        
        # FC layers with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create model
model = SimpleCNN().to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("🏗️ Simple CNN Architecture:")
print("="*60)
print(model)
print("="*60)
print(f"\n📊 Total parameters: {total_params:,}")
print(f"📊 Trainable parameters: {trainable_params:,}")

# Test forward pass
dummy_input = torch.randn(1, 1, 28, 28).to(device)
dummy_output = model(dummy_input)
print(f"\n✅ Input shape: {dummy_input.shape}")
print(f"✅ Output shape: {dummy_output.shape}")
print(f"\n💡 Output has 10 values (one for each class)!")

## 🚂 Part 4: Training and Evaluation Functions

Let's create reusable training functions!

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """
    Train model for one epoch
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(train_loader, desc='Training', leave=False):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def evaluate(model, test_loader, criterion, device):
    """
    Evaluate model on test set
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss = running_loss / len(test_loader)
    test_acc = 100. * correct / total
    
    return test_loss, test_acc

print("✅ Training functions defined!")
print("\n🚂 Ready to train models!")

## 🎯 Part 5: Train the Simple CNN

Let's train our first CNN and see how it performs!

In [ ]:
# Setup training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5

# Training history
train_losses, train_accs = [], []
test_losses, test_accs = [], []

print("🚂 Training Simple CNN...\n")
print("="*60)

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Test Loss:  {test_loss:.4f} | Test Acc:  {test_acc:.2f}%")
    print("-" * 60)

print("\n✅ Training complete!")
print(f"\n🎯 Final Test Accuracy: {test_accs[-1]:.2f}%")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(train_losses, 'b-o', label='Train Loss', linewidth=2)
ax1.plot(test_losses, 'r-o', label='Test Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax1.set_title('Training and Test Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Accuracy
ax2.plot(train_accs, 'b-o', label='Train Accuracy', linewidth=2)
ax2.plot(test_accs, 'r-o', label='Test Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
ax2.set_title('Training and Test Accuracy', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Training Progress:")
print(f"• Started at: {train_accs[0]:.2f}% (train), {test_accs[0]:.2f}% (test)")
print(f"• Ended at: {train_accs[-1]:.2f}% (train), {test_accs[-1]:.2f}% (test)")
print(f"• Improvement: +{train_accs[-1]-train_accs[0]:.2f}% (train), +{test_accs[-1]-test_accs[0]:.2f}% (test)")

## 📐 Part 6: Building VGG-Style Architecture

Let's implement a deeper CNN inspired by VGG!

In [ ]:
class VGGStyleCNN(nn.Module):
    def __init__(self):
        super(VGGStyleCNN, self).__init__()
        
        # VGG-style blocks: Conv-Conv-Pool pattern
        # Block 1: 1 → 64 channels
        self.conv1_1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        # Block 2: 64 → 128 channels
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        # Pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 10)
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Block 1: Conv-ReLU-Conv-ReLU-Pool
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)  # 28×28 → 14×14
        
        # Block 2: Conv-ReLU-Conv-ReLU-Pool
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)  # 14×14 → 7×7
        
        # Flatten and FC layers
        x = x.view(-1, 128 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create VGG-style model
vgg_model = VGGStyleCNN().to(device)

# Count parameters
vgg_params = sum(p.numel() for p in vgg_model.parameters())

print("📐 VGG-Style CNN Architecture:")
print("="*60)
print(vgg_model)
print("="*60)
print(f"\n📊 Total parameters: {vgg_params:,}")
print(f"📊 Simple CNN had: {total_params:,} parameters")
print(f"📊 VGG-style is {vgg_params/total_params:.2f}x larger!")

print("\n💡 VGG pattern: Two Conv layers before each Pool!")

## 🚀 Part 7: ResNet Skip Connections

Let's implement residual blocks with skip connections!

In [ ]:
class ResidualBlock(nn.Module):
    """
    Basic ResNet building block with skip connection
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        
        # Main path: Conv-BN-ReLU-Conv-BN
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Skip connection (shortcut)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            # If dimensions change, adjust skip connection
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, 
                         stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        # Main path
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        # Add skip connection
        out += self.shortcut(x)
        out = F.relu(out)
        
        return out

# Test residual block
test_block = ResidualBlock(64, 64).to(device)
test_input = torch.randn(1, 64, 28, 28).to(device)
test_output = test_block(test_input)

print("🚀 Residual Block Created!")
print(f"\n✅ Input shape: {test_input.shape}")
print(f"✅ Output shape: {test_output.shape}")
print(f"\n💡 Skip connection adds input directly to output!")
print(f"💡 This helps gradients flow backward easily!")

## 🎨 Part 8: Visualizing Learned Filters

Let's see what our trained CNN learned!

In [ ]:
# Get first conv layer weights
first_layer_weights = model.conv1.weight.data.cpu()

# Visualize 16 filters
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
axes = axes.flatten()

for i in range(16):
    filter_img = first_layer_weights[i, 0].numpy()
    axes[i].imshow(filter_img, cmap='gray')
    axes[i].set_title(f"Filter {i+1}", fontsize=9)
    axes[i].axis('off')

plt.suptitle("Learned Filters from First Convolutional Layer", 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🎨 What the filters learned:")
print("• Dark/light patterns for edge detection")
print("• Different orientations (vertical, horizontal, diagonal)")
print("• Some detect textures and gradients")
print("\n💡 These were learned automatically during training!")

## 🗺️ Part 9: Visualizing Feature Maps

Let's see how the CNN processes an image layer by layer!

In [ ]:
# Get a test image
test_img, test_label = test_dataset[0]
test_img_batch = test_img.unsqueeze(0).to(device)

# Hook to capture feature maps
activations = {}
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# Register hooks
model.conv1.register_forward_hook(get_activation('conv1'))
model.conv2.register_forward_hook(get_activation('conv2'))

# Forward pass
with torch.no_grad():
    output = model(test_img_batch)
    prediction = output.argmax(dim=1).item()

# Visualize
fig = plt.figure(figsize=(15, 8))

# Original image
ax1 = plt.subplot(3, 1, 1)
ax1.imshow(test_img.squeeze().numpy(), cmap='gray')
ax1.set_title(f"Input Image: {class_names[test_label]}\nPredicted: {class_names[prediction]}",
             fontsize=12, fontweight='bold')
ax1.axis('off')

# Conv1 feature maps (show 8 of 32)
conv1_features = activations['conv1'][0].cpu().numpy()
for i in range(8):
    ax = plt.subplot(3, 8, 8 + i + 1)
    ax.imshow(conv1_features[i], cmap='viridis')
    if i == 0:
        ax.set_ylabel('Conv1\n32 maps', fontsize=9, fontweight='bold')
    ax.set_title(f"Map {i+1}", fontsize=8)
    ax.axis('off')

# Conv2 feature maps (show 8 of 64)
conv2_features = activations['conv2'][0].cpu().numpy()
for i in range(8):
    ax = plt.subplot(3, 8, 16 + i + 1)
    ax.imshow(conv2_features[i], cmap='plasma')
    if i == 0:
        ax.set_ylabel('Conv2\n64 maps', fontsize=9, fontweight='bold')
    ax.set_title(f"Map {i+1}", fontsize=8)
    ax.axis('off')

plt.suptitle("Feature Maps Across CNN Layers", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🗺️ Feature Map Progression:")
print(f"• Input: {test_img.shape}")
print(f"• After Conv1: {activations['conv1'].shape}")
print(f"• After Conv2: {activations['conv2'].shape}")
print("\n💡 Notice: More channels, smaller spatial size!")

## 🎯 Challenge - Build Your Own CNN

**Your Task:** Create a custom CNN with:
1. At least 3 convolutional layers
2. Use both Max Pooling and Batch Normalization
3. 2-3 fully connected layers
4. Achieve >88% test accuracy on Fashion-MNIST

**Requirements:**
- Define the model class
- Train for 5 epochs
- Plot training curves
- Report final accuracy

**Hints:**
- Try 32 → 64 → 128 channel progression
- Use nn.BatchNorm2d after conv layers
- Learning rate: 0.001 with Adam optimizer

**Bonus:**
- Implement dropout
- Try data augmentation
- Visualize your model's feature maps


In [ ]:
# YOUR CODE HERE
# Build and train your custom CNN



## 📚 Summary & Key Takeaways

### 🎓 What We Learned Today:

**1. Pooling Layers:**
- Max pooling: Keeps strongest features
- Average pooling: Smooths output
- Reduces spatial dimensions (computational efficiency)
- Makes features translation-invariant

**2. Complete CNN Blocks:**
- Standard pattern: Conv → ReLU → Pool
- Can repeat: Conv → Conv → ReLU → Pool (VGG style)
- End with FC layers for classification

**3. Famous Architectures:**
- AlexNet (2012): Deep learning breakthrough
- VGG (2014): Simplicity with 3×3 filters
- ResNet (2015): Skip connections for very deep networks

**4. PyTorch Implementation:**
- nn.Conv2d for convolution
- nn.MaxPool2d for pooling
- nn.Linear for fully connected
- F.relu for activation

**5. Training Pipeline:**
- Load data with DataLoader
- Define model, loss, optimizer
- Training loop: forward, loss, backward, update
- Evaluate on test set

**6. Visualization:**
- Learned filters show edge detectors
- Feature maps reveal hierarchical learning
- Early layers: edges, later layers: complex patterns

### 🎯 Key Architecture Patterns:

**Simple CNN:**
```
Input → Conv → ReLU → Pool → Conv → ReLU → Pool → FC → Output
```

**VGG-Style:**
```
Input → [Conv → Conv → Pool]×N → FC → Output
Pattern: Double channels, halve dimensions
```

**ResNet-Style:**
```
Input → [Conv → Conv + Skip Connection]×N → FC → Output
Skip connection: Add input to output
```

### 💡 Pro Tips:

1. **Start simple** - Add complexity gradually
2. **Use 3×3 kernels** - Most effective size
3. **Double channels** after each pooling
4. **Batch normalization** helps training
5. **Dropout** prevents overfitting
6. **Adam optimizer** good default choice
7. **Learning rate 0.001** typical starting point
8. **Data augmentation** improves generalization

### 🚀 What's Next (Day 3):

**Tomorrow we'll cover:**
- Regularization techniques
- Dropout and batch normalization in detail
- Overfitting prevention
- TensorBoard for training visualization
- Advanced training tricks

---

**Excellent work! You can now build and train real CNNs! 🎉**

Remember: Modern CNNs are just clever combinations of these building blocks!